In [36]:
import re
import os
import time
# import config
from glob import glob

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

In [23]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 0
MODEL_NAME  = 'gpt-4o-mini'
EMBEDDING_NAME = 'text-embedding-3-large'

COLLECTION_NAME_PERSON = 'korean_history'
PERSIST_DIRECTORY_PERSON = 'vector_store/korean_history_db'
name = ["고대", "고려", "근대", "조선", "현대"]


In [24]:
# Split
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name=MODEL_NAME,
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

In [44]:
for i in name:
    file = glob(f"./pdf/사건/{i}/*.pdf")
    document_list = []
    for path in file:
        loader = PyMuPDFLoader(path)
        load_docs = loader.load()
        full_text = [doc.page_content for doc in load_docs] 
        full_text = ''.join(full_text)
        full_text = re.sub(r"관련사료", "", full_text)
        full_text = re.sub(r"\([一-龥]+\)", "", full_text)
        # full_text = re.sub(r"\n", "", full_text)

        docs = splitter.split_text(full_text)
        id = "_".join(os.path.splitext(os.path.basename(path))[0].split('_')[:2])
        title = os.path.splitext(os.path.basename(path))[0].split('_')[-1]
        metadata = {
        "id":id,
        "title": title,
        "full_text": full_text
        }

        for doc in docs:
            _doc = Document(metadata=metadata, page_content=doc)
            document_list.append(_doc)

    embedding_model = OpenAIEmbeddings(model=EMBEDDING_NAME)

    vector_store = Chroma.from_documents(
        documents=document_list,
        embedding=embedding_model,
        ids=[doc.metadata['title'] for doc in document_list],
        collection_name=COLLECTION_NAME_PERSON,
        persist_directory=PERSIST_DIRECTORY_PERSON
    )

print(len(document_list))

45


In [28]:
print(document_list[0])

page_content='고구려 건국
하늘의 자손, 나라를 세우다
기원전 37년 ~
 개요
고구려 건국의 주도세력은 압록강 중상류 지역에 살고 있던 주민 집단과 부여계의 남하 집단과
의 융합으로 이루어졌다. 또한 하천과 계곡을 기준으로 만들어진 ‘나’라고 하는 작은 정치 단
위들의 상호 협력과 정복 과정을 통해 국가를 형성하였다.
 고구려의 건국 설화
『삼국사기』 등에 전하는 건국 설화에 따르면 고구려를 세운 사람은 부여에서 남하한
주몽이다. 주몽의 아버지는 천제의 아들인 해모수이고, 어머니는 물을 다스
리는 하백의 딸 유화라고 전한다. 유화는 부모의 허락 없이 해모수와 정을 통하여 쫓
겨났다가 부여의 왕인 금와왕에게 거두어졌다. 부여 왕궁에 머물게 된 유화의 몸에는 햇
빛이 따라 다니며 비추었는데 이 때문에 임신이 되었다. 유화는 얼마 후 사람이 아닌 알을 낳았
고, 그 알을 깨고 나온 것이 바로 주몽이다. 주몽은 어렸을 때부터 활을 잘 쏘고 재주가 뛰어나
부여의 왕자 대소 등에게 시기의 대상이 되었다. 이에 부여에서 용납되지 못함을 알고, 자
신을 따르는 동료들과 함께 남쪽으로 도망하였다.
주몽은 기원전 37년 졸본천에 도읍을 정하고 고구려를 건국하였다. 원래 이 지역에 자리
를 잡고 있던 비류국의 송양왕과 경쟁 관계를 형성하기도 하였으나 타고난 재주
와 권능으로 제압하였고, 이후 주변국들에 대한 정복 활동을 통해 나라의 기틀을 다졌다. 기원전
19년(동명성왕 19) 부여에 남겨 두고 왔던 부인과 함께 아들 유리가 찾아오자 태자로 임명
하였고 같은 해에 세상을 떠났다. 사후에는 동명성왕이라 일컬어지게 되었다.
 고구려의 종족적 계통
고구려 건국 설화의 내용은 상당 부분 부여의 건국 설화인 동명 설화를 차용한 것이다. 따라서
그 내용을 있는 그대로 역사적 사실이라 믿기는 어렵다. 하지만 한편으로는 전혀 상관이 없는 집
단이 부여의 건국 설화를 차용해 사용했다는 것도 생각하기 힘들다. 부여계 집단이 남하하여 고
구려라는 정치 집단을 형성하는 데 참여한 것은 사

In [17]:
vector_store._collection.count()

88

In [45]:
result = vector_store.similarity_search_with_score(
    query="이순신이 참전한 전투가 뭐야",
    k=3
)
result

[(Document(metadata={'full_text': '한산도 해전\n일본의 수륙병진 전략을 무력화하\n여 전쟁의 판세를 뒤엎다\n1592년(선종 25) ~\n 임진왜란의 발발과 해전의 부재\n조선 조정은 임진왜란이 발생하기 직전에 남해안의 수비 태세를 강화할 새로운 인재를 발탁하였\n다. 이 시기에 이순신과 이억기가 전라좌수사와 우수사로 임명되어 임란 때까지 약 1년\n남짓 나름대로 전쟁 준비를 할 수 있었다. \n그러나 임진왜란 발발 초기 경상도의 전황은 좋지 못하였다. 다대포첨사 윤흥신과 부산\n첨사 정발처럼 맞서 싸우다 죽음에 이른 경우도 있었던 반면, \n 경상도\n좌병사 이각, 경상좌수사 박홍처럼 싸우지도 않고 달아난 경우도 있어 \n 일\n본군이 쉽게 전쟁을 주도하였다. 경상우수사 원균도 부임한 지 얼마 되지 않은 탓도 있었으나 전\n선 4척만을 지켰을 뿐 휘하세력을 결집하는 데 실패하였다. \n임진왜란 발발 후 한동안 원균이 일본 군선 10척을 분멸한 것 \n 을 제외하면 양국 수군\n이 해전을 벌였다는 기록이 없다. 이는 일본군의 전략과 관련이 된 것으로 전국시대를 거치면서\n많은 전쟁을 경험한 반면 해전의 경험은 많지 않았기 때문에 처음부터 일본 수군의 목적은 조선\n수군과의 전투가 아니라 병력 및 군수품 운반이나 상대방 보급로 차단이었다.\n이와 같이, 일본 수군은 경상좌수영과 주변 진영을 점령한 후 조선 수군과의 해전을 추구하지 않\n았는데, 이것은 원균에게는 전황을 살피면서 구원을 요청할 수 있는 기회를 주었고, 이순신으로\n하여금 휘하의 수군 세력을 단속하고 결집할 시간을 벌어주었다. 이순신은 이 기간을 이용하여\n지자총통, 현자총통을 시험 발사하거나, 수군을 모으고, 정찰 활동을 펼치고, 휘하 함대를 집결하\n였다.\n 이순신의 1, 2차 출전\n4월 30일 원균에게서 경상우수영이 점령되었다는 공문이 오고, 인접한 경상우도 진영 네 군데의\n장수와 병력이 흩어져버렸다는 소식이 이순신에게 전해졌다. 출전 준비를 해오던 이순신은 

In [ ]:
retriever = vector_store.as_retriever(search_type="mmr")

In [ ]:
query = "장영실에 대해 알려줘"
result = retriever.invoke(query)
result